### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [10]:
### OpenAI API Key and Open Source models -- Llama3, Gemma2, mistrial -- Groq
import os
from dotenv import load_dotenv

load_dotenv()

import openai
openai.api_key = os.getenv("OPENAI_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

In [11]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
model = ChatGroq(model="gemma2-9b-it", groq_api_key=groq_api_key)
model


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x116969050>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x117502bd0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [12]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result = model.invoke(messages)

In [13]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
parser.invoke(result)

"Bonjour \nComment allez-vous? \n\n\nLet me know if you'd like more translations!\n"

In [14]:
### Using LCEL - chain the components
chain = model | parser

chain.invoke(messages)

'Bonjour, comment allez-vous ? \n\n\nLet me know if you need any other phrases translated!\n'

In [15]:
### Prompt Template
from langchain_core.prompts import ChatPromptTemplate

generic_template = "Translate the following into {language}:"
prompt = ChatPromptTemplate.from_messages(
    [("system", generic_template), ("user", "{text}")]
)

In [17]:
result = prompt.invoke({"language": "French", "text": "Hello"})

In [18]:
result.to_messages()

[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [19]:
### Chaining together components with LCEL
chain = prompt | model | parser

chain.invoke({"language": "French", "text": "Hello"})

'Bonjour  \n'